## 파이토치 기초(4) - CNN

* day 1 - 텐서와 Autograd : https://dacon.io/codeshare/4478
* day 2 - 신경망모델 구현하기 : https://dacon.io/codeshare/4495
* day 3 - DNN : https://dacon.io/codeshare/4532
* 오늘은 영상처리에 탁월한 성능을 자랑하는 CNN의 원리를 알아보고, Fashion MNIST에 적용해본 후, CNN을 이용한 ResNet모델로 좀 더 복잡한 컬러 이미지까지 다뤄보겠습니다.

* 컴퓨터에서 보는 모든 이미지는 픽셀값들을 가로, 세로로 늘어놓은 행렬로 표현할 수 있습니다.
* 컨볼루션은 계층적으로 이미지를 인식하 수 있도록 단계마다 이미지의 특징을 추출하는 것을 말합니다.
* CNN은 이미지를 추출하는 필터로 Convolution Neural Network 즉 컨볼루션을 하는 인공 신경망입니다.

* CNN 모델은 일반적으로 컨볼루션 계층(convolution layer), 풀링 계층(pooling layer), 특징들을 모아 최종 분류하는 일반적인 인공신경망 계층으로 구성됩니다.
* 컨볼루션을 거쳐 만들어진 새로운 이미지는 특징 맵(feature map)이라고도 불립니다. 컨볼루션 계층마다 여러 특징 맵들이 만들어지며, 다음 단계인 풀링(pooling) 계층으로 넘어가게 됩니다. 컨볼루션 계층과 풀링 계층을 여러 겹 쌓아, 각 단계에서 만들어진 특징 맵을 관찰하면 CNN 모델이 이미지를 계층적으로 인식하는 것을 볼 수 있습니다.
* 특징 맵의 크기가 크면 학습이 어렵고 과적합의 위험이 증가합니다.

### CNN 모델 구현하기

* 여러 CNN 모델은 컨볼루션, 풀링, 드롭아웃, 그리고 일반적인 신경망 계층의 조합으로 이루어집니다.
* 컨볼루션 -> 풀링 -> 컨볼루션 -> 드롭아웃 -> 풀링 -> 신경망 -> 드롭아웃 -> 신경망의 예제를 구현해보겠습니다.
* 일반 인공신경망을 CNN 계층으로 대체하면 되기 때문에 전체적 구현은 day-3의 신경망 구현법과 매우 비슷합니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
#USE_CUDA는 CUDA를 사용할 수 있는지 확인하는 코드이고, DEVICE는 USE_CUDA의 결과에 따라 cpu를 쓸지 gpu를 쓸지 결정합니다.

In [3]:
# 이폭과 배치크기를 정해줍니다
EPOCHS     = 40
BATCH_SIZE = 64

In [4]:
# Fashion MNIST 데이터셋을 불러옵니다.
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([ #transforms를 이용한 전처리는 파이토치 텐서화와 정규화만 하였습니다.
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./.data\MNIST\raw\train-images-idx3-ubyte.gz to ./.data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./.data\MNIST\raw\train-labels-idx1-ubyte.gz to ./.data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./.data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./.data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./.data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./.data\MNIST\raw



In [5]:
# 학습하기
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) #만드는 모델의 커널크기는 5x5입니다. 숫자를 하나만 지정하면 정사각형으로 간주합니다
        # nn.Conv2d는 입력 x를 받는 함수를 반환합니다. 
        # nn.Conv2d의 첫 두 파라미터는 입력 채널수(in_channels)와 출력 채널수(out_channels)입니다.
        # 첫 컨볼루션 계층에서는(self.conv1) 10개의 특징맵을 생성합니다
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        # 두번째 컨볼루션 게층에서는 10개의 특징맵을 받아 20개의 특징맵을 만듭니다.
        self.conv2_drop = nn.Dropout2d() 
        # 컨볼루션 결과 출력값에는 드롭아웃을 해줍니다. nn.Dropout2d 모듈로 드롭아웃 인스턴스를 만들 수 있습니다.
        self.fc1 = nn.Linear(320, 50) # 컨볼루션과 드롭아웃을 거친 이미지는 nn.Linear의 일반 신경망을 거칩니다.
        self.fc2 = nn.Linear(50, 10) # 입력크기 50, 출력은 분류할 클래스 개수인 10으로 설정합니다. (각 계층의 출력크기는 임의로 지정)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        # 입력 받은 것이 첫 컨볼루션 계층을 거치고 F.max_pool2d함수를 거치게 합니다.
        # F.max_pool2d의 두 번째 입력은 커널 크기입니다.(2)
        # 컨볼루션과 맥스 풀링을 통과한 x는 F.relu()활성화 함수를 거칩니다
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2)) # 두번째 컨볼루션 계층도 똑같이 반복합니다
        x = x.view(-1, 320) # 컨볼루션 계층 2개를 거쳐 특징맵이 된 x를 1차원으로 펴줍니다. (-1은 남는차원 모두, 320은 x가 가진 원소개수)
        x = F.relu(self.fc1(x)) 
        x = F.dropout(x, training=self.training) # ReLU 활성화 함수를 거친 뒤 드롭아웃을 사용합니다
        x = self.fc2(x) # 0부터 9까지 레이블을 갖는 10개의 출력값을 가지는 신경망
        return x

In [6]:
# 파라미터를 지정합니다.
model     = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5) #최적화 알고리즘으로 파이토치에 내장되어 있는 optim.SGD를 사용합니다

In [7]:
# 앞 장에서 본 과정과 완전히 동일한 모델 훈련과 평가코드입니다.
# 훈련코드
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# 성능확인코드
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [8]:
# 코드 실행
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy)) # day3에서와 다르게 정확도가 99%까지 올라간 것을 볼 수 있습니다

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303096
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.018277
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.267298
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.469899
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.620442
[1] Test Loss: 0.2143, Accuracy: 93.83%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.500467
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.467868
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.372048
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.184066
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.389603
[2] Test Loss: 0.1278, Accuracy: 96.18%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.127458
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.555271
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.164062
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.325616
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.478401
[3] Test Loss: 0.0970, Accuracy: 97.18%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.252095
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.177119
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.286993


Train Epoch: 29 [38400/60000 (64%)]	Loss: 0.117330
Train Epoch: 29 [51200/60000 (85%)]	Loss: 0.162402
[29] Test Loss: 0.0358, Accuracy: 98.90%
Train Epoch: 30 [0/60000 (0%)]	Loss: 0.369476
Train Epoch: 30 [12800/60000 (21%)]	Loss: 0.072453
Train Epoch: 30 [25600/60000 (43%)]	Loss: 0.279839
Train Epoch: 30 [38400/60000 (64%)]	Loss: 0.031995
Train Epoch: 30 [51200/60000 (85%)]	Loss: 0.071114
[30] Test Loss: 0.0342, Accuracy: 98.99%
Train Epoch: 31 [0/60000 (0%)]	Loss: 0.075928
Train Epoch: 31 [12800/60000 (21%)]	Loss: 0.102470
Train Epoch: 31 [25600/60000 (43%)]	Loss: 0.084451
Train Epoch: 31 [38400/60000 (64%)]	Loss: 0.055920
Train Epoch: 31 [51200/60000 (85%)]	Loss: 0.218221
[31] Test Loss: 0.0339, Accuracy: 98.93%
Train Epoch: 32 [0/60000 (0%)]	Loss: 0.287325
Train Epoch: 32 [12800/60000 (21%)]	Loss: 0.148126
Train Epoch: 32 [25600/60000 (43%)]	Loss: 0.083411
Train Epoch: 32 [38400/60000 (64%)]	Loss: 0.216294
Train Epoch: 32 [51200/60000 (85%)]	Loss: 0.149540
[32] Test Loss: 0.0322, A

### ResNet으로 컬러 데이터셋에 적용하기

* ResNet은 이미지넷 대회에서 2015년에 우승한 모델로, 신경망을 깊게 쌓으면 오히려 성능이 나빠지는 문제를 해결하는 방법을 제시했고, 이후 DenseNet 등의 파생모델에 영향을 주었습니다.
* 이번에는 Fashion MNIST 대신 CIFAR-10 데이터셋을 사용하겠습니다. CIFAR-10 데이터셋은 32x32 크기의 컬러 이미지 6만개를 포함하고 있으며 자동차, 새, 고양이, 사슴 등 10가지 분류가 존재합니다.
* 컬러 이미지의 픽셀값은 몇 가지 채널(channel)로 구성되는데, 채널이란 이미지 구성요소를 가리킵니다. 오늘날 가장 널리 쓰이는 24bit 컬러 이미지는 R,G,B 각각에 8bit(0~255)씩 색상값을 가지는 3가지 채널을 사용합니다.

In [9]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data', #CIFRA-10 데이터를 불러옵니다.
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.RandomCrop(32, padding=4),
                       transforms.RandomHorizontalFlip(), # 과적합 방지를 위해 RandomCrop과 RandomHorizontalFlip을 추가했습니다
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./.data\cifar-10-python.tar.gz to ./.data


* ResNet은 여러 단계의 신경망을 거치며 최초 입력 이미지에 대한 정보가 소실되는 문제를 해결하는 방안을 제시합니다.
* ResNet은 네트워크를 작은 블록인 Residual 블록으로 나누어 Residual 블록의 출력에 입력이었던 x를 더함으로써 모델을 훨씬 깊게 설계할 수 있도록 했습니다.
* 입력과 출력의 관계를 바로 학습하기보다 입력과 출력의 차이를 따로 학습하는 것이 성능이 좋다는 가설입니다.

In [10]:
# 모델만들기

class BasicBlock(nn.Module): # Residual 블록을 BasicBlock이라는 새로운 파이토치 모듈로 정의해서 사용합니다.
    # 파이토치는 nn.Module을 이용하여 모듈위에 또다른 모듈을 쌓아 올릴 수 있습니다.
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        # nn.BatchNorm2d는 배치 정규화(batch normalization)을 수행하는 계층입니다.
        # 학습률을 너무 높게 잡았을 때 기울기가 소실되거나 발산하는 증상을 예방하여 학습 과정을 안정화합니다.
        # 즉, 이 계층은 자체적으로 정규화를 수행해 드롭아웃과 같은 효과를 내는 장점이 있습니다.
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

    # ResNet의 두번째 블록 부터는 in_planes()를 받아 self.bn2 계층과 출력크기가 같은 planes를 더해주는 self.shortcut모듈을 정의합니다
        self.shortcut = nn.Sequential() # nn.Sequential()은 여러 모듈(nn.Module)을 하나의 모듈로 엮는 역할을 합니다.
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x): #데이터의 흐름은 앞서 한 설명과 같습니다.
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

# 모델정의
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16 # self.in_planes 변수는 _make_layer 함수가 층을 만들 때 채널 출력값을 기록하는 데 쓰입니다.
        # layer1이 입력받는 채널의 개수가 16개이므로 16으로 초기화해줍니다.

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride=1) 
        #self._make_layer()는 nn.Sequential의 도구로 여러 BasicBlock 모듈을 하나로 묶어주는 역할을 합니다.
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        # self.layer1~3은 컨볼루션 계층과 마찬가지로 모듈(nn.Module)로 취급하면 됩니다.
        self.linear = nn.Linear(64, num_classes)

    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    # _make_layer 함수는 self.in_planes 채널 개수로부터 직접 입력받은 인수인 planes 채널 개수만큼을 출력하는 BasicBlock을 생성합니다.
    # layer1 : 16채널에서 16채널을 보내는 BasicBlock 2개
    # layer2 : 16채널을 받아 32채널을 출력하는 BasicBlock 1개와 32채널에서 32채널을 내보내는 BasicBlock 1개
    # layer3 : 32채널을 받아 64채널을 출력하는 BasicBlock 1개와 64채널에서 64채널을 출력하는 BasicBlock 1개

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
    # ResNet 모델은 위와같이 컨볼루션, 배치정규화, 활성화 함수를 통과하고 사전에 정의해둔 BasicBlock 층을 가지고 있는
    # layer1, layer2, layer3를 통과하게 됩니다. 각 layer는 2개의 Residual 블록을 갖고 있고 
    # 이렇게 나온 값에 평균 풀링을 하고 마지막 계층을 거쳐 분류결과를 출력합니다.

* 이번 예제에서는 학습률 감소(learning rate decay)기법을 사용합니다.
* 학습률 감소는 학습이 진행하면서 최적화 함수의 학습률을 점점 낮춰서 더 정교하게 최적화합니다. 
* 이는 파이토치 내부의 optim.lr_scheduler.StepLR 도구로 적용할 수 있습니다.

In [11]:
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=0.0005) 
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
# scheduler는 이폭마다 호출되며 step_size를 50으로 지정해 50번 호출될 때 학습률에 0.1(gamma)만큼 곱합니다.
# 즉 0.1로 시작한 학습률은 50이폭 이후에 0.1 x 0.1 = 0.01로 낮아집니다.

In [12]:
#print(model) #처음부터 끝까지 모든 계층이 어떻게 생겼는지 print(model)로 볼 수 있습니다.

In [13]:
# 훈련하기 (코드 돌리기)
for epoch in range(1, EPOCHS + 1):
    scheduler.step() #코드는 위와 대부분 동일하나 scheduler.step()으로 학습률을 조금 낮춰주는 단계가 추가되었습니다.
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

C:\Users\syi06\.conda\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.559069
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.842535
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.675769
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.403505
[1] Test Loss: 1.5372, Accuracy: 48.17%
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.265703
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.304142
Train Epoch: 2 [25600/50000 (51%)]	Loss: 0.890852
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.127497
[2] Test Loss: 1.0901, Accuracy: 62.10%
Train Epoch: 3 [0/50000 (0%)]	Loss: 1.010721
Train Epoch: 3 [12800/50000 (26%)]	Loss: 0.851191
Train Epoch: 3 [25600/50000 (51%)]	Loss: 0.896181
Train Epoch: 3 [38400/50000 (77%)]	Loss: 0.755468
[3] Test Loss: 1.0075, Accuracy: 65.59%
Train Epoch: 4 [0/50000 (0%)]	Loss: 0.967926
Train Epoch: 4 [12800/50000 (26%)]	Loss: 0.845373
Train Epoch: 4 [25600/50000 (51%)]	Loss: 0.778225
Train Epoch: 4 [38400/50000 (77%)]	Loss: 1.230998
[4] Test Loss: 1.1727, Accuracy: 61.22%
Train Epoch: 5 [0/50000 (0%)]	Loss: 1.105525
Train Epoch: 5 

Train Epoch: 35 [25600/50000 (51%)]	Loss: 0.520221
Train Epoch: 35 [38400/50000 (77%)]	Loss: 0.527956
[35] Test Loss: 0.9450, Accuracy: 71.38%
Train Epoch: 36 [0/50000 (0%)]	Loss: 0.575580
Train Epoch: 36 [12800/50000 (26%)]	Loss: 0.529518
Train Epoch: 36 [25600/50000 (51%)]	Loss: 0.709076
Train Epoch: 36 [38400/50000 (77%)]	Loss: 0.568758
[36] Test Loss: 0.8203, Accuracy: 71.90%
Train Epoch: 37 [0/50000 (0%)]	Loss: 0.406103
Train Epoch: 37 [12800/50000 (26%)]	Loss: 0.594710
Train Epoch: 37 [25600/50000 (51%)]	Loss: 0.374258
Train Epoch: 37 [38400/50000 (77%)]	Loss: 0.765149
[37] Test Loss: 0.7800, Accuracy: 74.44%
Train Epoch: 38 [0/50000 (0%)]	Loss: 0.776516
Train Epoch: 38 [12800/50000 (26%)]	Loss: 0.512619
Train Epoch: 38 [25600/50000 (51%)]	Loss: 0.468177
Train Epoch: 38 [38400/50000 (77%)]	Loss: 0.668412
[38] Test Loss: 1.3345, Accuracy: 65.05%
Train Epoch: 39 [0/50000 (0%)]	Loss: 0.736510
Train Epoch: 39 [12800/50000 (26%)]	Loss: 0.506434
Train Epoch: 39 [25600/50000 (51%)]	Loss